# Convert snuffler_files to files which can be opened by snuffler

In [1]:
import pandas as pd
import numpy as np

In [44]:
df = pd.read_csv('../snuffler_picks2.txt', names=['sd', 'st', 'ed', 'et', 'dt', 'bla', 'class', 'stat'], skiprows=1)
df

,sd,st,ed,et,dt,bla,class,stat
0,2019-05-17,23:41:39.9690,2019-05-17,23:42:22.7896,42.82059979438782,NaN,3.0,XP.ILL12..EHZ
1,2019-05-17,23:41:41.4662,2019-05-17,23:42:32.2720,50.80579996109009,NaN,3.0,XP.ILL13..EHZ
2,2019-05-17,09:26:52.5869,2019-05-17,09:27:40.7454,48.15849995613098,NaN,3.0,XP.ILL14..EHZ
3,2019-05-17,07:00:34.1965,2019-05-17,07:01:03.6467,29.450199842453003,NaN,4.0,XP.ILL17..EHZ
4,2019-05-17,07:00:37.9249,2019-05-17,07:01:06.9365,29.011600017547607,NaN,4.0,XP.ILL18..EHZ
5,2019-05-17,07:00:38.2744,2019-05-17,07:01:06.8666,28.59220004081726,NaN,4.0,XP.ILL18..EHZ
6,2019-05-17,07:00:40.3017,2019-05-17,07:01:07.4259,27.124199867248535,NaN,4.0,XP.ILL18..EHZ
7,2019-05-17,10:02:57.8257,NaN,0,XP.ILL07..EHZ,NaN,NaN,NaN
8,2019-05-17,10:05:53.4261,2019-05-17,10:07:03.4020,69.97589993476868,NaN,1.0,None
9,2019-05-17,10:08:39.0653,NaN,0,XP.ILL07..EHZ,NaN,NaN,NaN


In [ ]:
df.to_csv('picks_for_snuffler', sep=' ', )

In [2]:
df_original = pd.read_csv('../snuffler_files/event_catalog_man_cont_hSNR_v4.txt')
df = df_original.copy()
df['class'] = df['class'].map('{:.0f}'.format)
df = df.astype(str)
df = df.replace('nan', '')
#df['class'] = df['class'].astype(str)
df.head()

,startdate,starttime,enddate,endtime,duration,bla,class,network
0,2017-05-24,01:43:28.3030,2017-05-24,01:43:40.9140,12.611000061035156,,1,XP.ILL08..EHZ
1,2017-05-24,01:43:28.5680,2017-05-24,01:43:41.5545,12.986500024795532,,1,XP.ILL04..EHZ
2,2017-05-24,01:43:28.5680,2017-05-24,01:43:41.7975,13.229499816894531,,1,XP.ILL01..EHZ
3,2017-05-24,01:43:28.8110,2017-05-24,01:43:41.7312,12.920199871063232,,1,XP.ILL03..EHZ
4,2017-05-24,01:43:29.0539,2017-05-24,01:43:41.5103,12.456399917602539,,1,XP.ILL02..EHZ


In [3]:
with open('../snuffler_files/picks_for_snuffler_hSNR_v4.txt', 'w') as file:
    file.write("# Snuffler Markers File Version 0.2\n")

df.to_csv('../snuffler_files/picks_for_snuffler_hSNR_v4.txt', sep=' ', header=False, index=False, mode='a')

# Prepare detections for snuffler (convert datetime to file which can be opened by snuffler)

This part needs a detection file, and saves a file in the detection folder beginning with snuffler_

In [1]:
import pandas as pd
import numpy as np
import obspy

In [2]:
# load detection file
year = 2021
plot_ending = 'test_v4_bigsf_clf2'
df = pd.read_csv(f'../detections/{year}/{year}_{plot_ending}.txt')
df.head()

,datetimes,classes
0,2021-06-23 00:00:20.000,0
1,2021-06-23 00:00:30.000,0
2,2021-06-23 00:00:40.000,0
3,2021-06-23 00:00:50.000,0
4,2021-06-23 00:01:00.000,0


In [3]:
# drop times without detections
mask = (df['classes'] == 1) # mask only detections (1 = True)
df_mask = df[mask] # drop all times without detection
df_mask

,datetimes,classes
221,2021-06-23 00:37:10.000,1
222,2021-06-23 00:37:20.000,1
223,2021-06-23 00:37:30.000,1
224,2021-06-23 00:37:40.000,1
225,2021-06-23 00:37:50.000,1
...,...,...
739913,2021-09-16 16:07:43.770,1
739914,2021-09-16 16:07:53.770,1
739915,2021-09-16 16:08:03.770,1
739916,2021-09-16 16:08:13.770,1


In [4]:
#df_new = pd.DataFrame(index=range(0,len(df_mask)), columns=['start', 'end', 'class', 'station'])
startdates = []
starttimes = []
enddates = []
endtimes = []
dts = []

short_sdates = []
short_stimes = []

count = 0
end = False
idx_list = df_mask.index.to_list()

for i,idx in enumerate(idx_list[:-1]):
    
    if idx == idx_list[i+1]-1: # idx < endtime
        if count == 0: # define starttime
            starttime = obspy.UTCDateTime(df_mask['datetimes'].loc[idx])
            startdates.append(starttime.date)
            starttimes.append(starttime.time)
        endtime = obspy.UTCDateTime(df_mask['datetimes'].iloc[i+1])
        count += 1
        if idx == idx_list[-2]:# append endtime and duration of last idx if event longer than one time window
            enddates.append(endtime.date)
            endtimes.append(endtime.time)
            dt = endtime - starttime
            dts.append(dt)
            end = True
            
    if idx != df_mask.index[i+1]-1: # idx bei endtime -> schreibe event
        if count == 0:
            starttime = obspy.UTCDateTime(df_mask['datetimes'].loc[idx])
            short_sdates.append(starttime.date)
            short_stimes.append(starttime.time)
        if count > 0:
            enddates.append(endtime.date)
            endtimes.append(endtime.time)
            dt = endtime - starttime
            dts.append(dt)
            count = 0

if end == False: # add last starttime if event only one time window long
    starttime = obspy.UTCDateTime(df_mask['datetimes'].iloc[-1])
    short_sdates.append(starttime.date)
    short_stimes.append(starttime.time)

In [5]:
print(len(starttimes))
print(len(startdates))
print(len(endtimes))
print(len(dts))
print(len(short_stimes))

1307
1307
1307
1307
468


In [6]:
df_long = pd.DataFrame()
df_long['sdate'] = startdates
df_long['stime'] = starttimes
df_long['edate'] = enddates
df_long['etime'] = endtimes
df_long['dt'] = dts
df_long['class'] = [0]*(len(dts))
df_long['station'] = ['None'] *len(dts)
df_long

,sdate,stime,edate,etime,dt,class,station
0,2021-06-23,00:37:10,2021-06-23,00:38:40,90.0,0,None
1,2021-06-23,03:34:10,2021-06-23,03:35:10,60.0,0,None
2,2021-06-23,05:13:40,2021-06-23,05:14:10,30.0,0,None
3,2021-06-23,14:56:50,2021-06-23,14:57:10,20.0,0,None
4,2021-06-24,01:15:10,2021-06-24,01:15:40,30.0,0,None
...,...,...,...,...,...,...,...
1302,2021-09-15,13:23:13.590000,2021-09-15,13:23:53.590000,40.0,0,None
1303,2021-09-15,18:23:33.590000,2021-09-15,18:24:03.590000,30.0,0,None
1304,2021-09-15,18:30:33.590000,2021-09-15,18:31:33.590000,60.0,0,None
1305,2021-09-16,07:18:23.770000,2021-09-16,07:18:53.770000,30.0,0,None


In [7]:
df_short = pd.DataFrame()
df_short['sdate'] = short_sdates
df_short['stime'] = short_stimes
df_short['class'] = [0]*(len(short_stimes))
df_short['station'] = ['None'] *len(short_stimes)
df_short

,sdate,stime,class,station
0,2021-06-23,16:56:00,0,None
1,2021-06-23,22:05:30,0,None
2,2021-06-24,02:26:10,0,None
3,2021-06-24,14:13:40,0,None
4,2021-06-24,14:19:30,0,None
...,...,...,...,...
463,2021-09-12,15:55:11.510000,0,None
464,2021-09-14,02:17:11.110000,0,None
465,2021-09-14,03:01:01.110000,0,None
466,2021-09-15,10:20:33.590000,0,None


In [8]:
with open(f'../detections/{year}/snuffler_{year}_{plot_ending}.txt', 'w') as file:
    file.write("# Snuffler Markers File Version 0.2\n")

df_long.to_csv(f'../detections/{year}/snuffler_{year}_{plot_ending}.txt', sep=' ', header=False, index=False, mode='a')
df_short.to_csv(f'../detections/{year}/snuffler_{year}_{plot_ending}.txt', sep=' ', header=False, index=False, mode='a')

In [28]:
# split date and time
df_time = df_mask['datetimes'].str.split(' ',expand=True).rename(columns={0:'date',1:'time'})
df_time.head()

,date,time
5066,2018-06-27,14:04:44.000
5067,2018-06-27,14:04:54.000
6851,2018-06-27,19:02:14.000
6854,2018-06-27,19:02:44.000
6996,2018-06-27,19:26:24.000


In [58]:
df_mask.iloc[0]

datetimes    2018-06-27 14:04:44.000
classes                            1
Name: 5066, dtype: object

In [82]:
from datetime import datetime
st = obspy.UTCDateTime(df_mask['datetimes'].iloc[1])
et = obspy.UTCDateTime(df_mask['datetimes'].iloc[2])
ddtt = et - st
print(st)
print(et)
print(ddtt)

2018-06-27T14:04:54.000000Z
2018-06-27T19:02:14.000000Z
17840.0


In [77]:
import obspy
obspy.UTCDateTime(df_mask['datetimes'].iloc[1+1])

2018-06-27T19:02:14.000000Z

In [57]:
df_mask['datetimes'].iloc[0+1]

'2018-06-27 14:04:54.000'